In [1]:
# Install packages
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow==2.18.0
!pip install opencv-python
!pip install tf-models-official==2.18.0
!pip install keras==3.8.0

  Using cached numpy-2.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-model-optimization 0.8.0 requires numpy~=1.23, but you have numpy 2.0.2 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency 

In [2]:
# Run imports
import os
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import PIL
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tqdm
import absl.logging

In [3]:
from google.colab import drive
drive.mount('/content/drive')

DATASET_PATH = '/content/drive/MyDrive/dataset'
TRAIN_DIR = os.path.join(DATASET_PATH, 'training')
TEST_DIR = os.path.join(DATASET_PATH, 'testing')

def get_video_paths_and_labels(base_path):
    paths = []
    labels = []
    label_map = {'passive': 0, 'active': 1}

    for label_name, label_value in label_map.items():
        folder = os.path.join(base_path, label_name)
        for fname in os.listdir(folder):
            if fname.endswith('.mp4'):
                paths.append(os.path.join(folder, fname))
                labels.append(label_value)

    return paths, labels

train_paths, train_labels = get_video_paths_and_labels('/content/drive/MyDrive/dataset/training')
test_paths, test_labels = get_video_paths_and_labels('/content/drive/MyDrive/dataset/testing')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Paramètres
frame_stride = 2
num_frames = 16
resolution = 224
num_classes = 2
batch_size = 4

# Fonction utilisée via tf.py_function
def read_video_opencv(path):
    path = path.decode() if isinstance(path, bytes) else path
    cap = cv2.VideoCapture(path)
    frames = []
    success = True

    while success:
        success, frame = cap.read()
        if success:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    cap.release()

    frames = frames[::frame_stride]
    frames = [cv2.resize(f, (resolution, resolution)) for f in frames]

    if len(frames) >= num_frames:
        frames = frames[:num_frames]
    else:
        pad = [np.zeros((resolution, resolution, 3), dtype=np.uint8) for _ in range(num_frames - len(frames))]
        frames += pad

    video = np.stack(frames, axis=0).astype(np.float32) / 255.0
    return video

def format_features_opencv(path, label):
    # Explicitly run tf.py_function on the CPU, ensure data types match.
    video = tf.numpy_function(read_video_opencv, [path], tf.float32)
    # Set the shape explicitly
    video.set_shape([num_frames, resolution, resolution, 3])
    label = tf.one_hot(label, num_classes)
    return video, label

def make_dataset(paths, labels, training=True):
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        dataset = dataset.shuffle(100)
    # Apply dataset.map() outside the strategy.scope
    dataset = dataset.map(
        # Add tf.function for AutoGraph
        tf.function(format_features_opencv), # This will ensure AutoGraph is enabled
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    # Add a tf.function decorator to format_features_opencv for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [24]:
train_dataset = make_dataset(train_paths, train_labels, training=True)
test_dataset = make_dataset(test_paths, test_labels, training=False)

In [25]:
from official.vision.configs import video_classification
from official.projects.movinet.configs import movinet as movinet_configs
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_layers
from official.projects.movinet.modeling import movinet_model

In [26]:
model_id = 'a5'

tf.compat.v1.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([1, 1, 1, 1, 3])

# Load pretrained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base.tar.gz -O movinet_a5_base.tar.gz -q
!tar -xvf movinet_a5_base.tar.gz

checkpoint_dir = 'movinet_a0_base'
checkpoint_path = tf.compat.v1.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.compat.v1.train.Checkpoint(model=model)

movinet_a5_base/
movinet_a5_base/checkpoint
movinet_a5_base/ckpt-1.data-00000-of-00001
movinet_a5_base/ckpt-1.index


In [37]:
import keras

epochs = 3

train_steps = len(train_paths) // batch_size
total_train_steps = train_steps * epochs

# Remove the @tf.function decorator from train_model
def train_model():
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    results = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=test_dataset,
        verbose=2)
    return results

# Call the function to train the model
results = train_model()

ValueError: The graph <tensorflow.python.framework.ops.Graph object at 0x7d510566a2c0> of the iterator is different from the graph <tensorflow.python.framework.ops.Graph object at 0x7d50ebfe8f40> the dataset: Tensor("BatchDatasetV2:0", shape=(), dtype=variant) was created in. Make sure that the dataset is created in the same graph as the iterator.